<a href="https://colab.research.google.com/github/vasudevgupta7/bigbird-intuition/blob/main/notebooks/bigbird_narrativeqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!pip install datasets
!pip install git+https://github.com/vasudevgupta7/transformers.git@add_big_bird

  Cloning https://github.com/vasudevgupta7/transformers.git (to revision add_big_bird) to /tmp/pip-req-build-1itfwjha
  Running command git clone -q https://github.com/vasudevgupta7/transformers.git /tmp/pip-req-build-1itfwjha
  Running command git checkout -b add_big_bird --track origin/add_big_bird
  Switched to a new branch 'add_big_bird'
  Branch 'add_big_bird' set up to track remote branch 'add_big_bird' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.4.0.dev0-cp37-none-any.whl size=2002859 sha256=f892cedb64be8a1b96d247e7c62c36d8bfeaaada529cfa1d429be0cfd487c971
  Stored in directory: /tmp/pip-ephem-wheel-cache-xs2f9qdf/wheels/43/e2/77/3201d9ec554567c33afa81d18640753e8c880f8cb7dfc49c2f
Successfully built transformers
  Found existing installation: transformers 4.4.1
    Uninstalling transformers-4.4.1:
      Successfully uninstalled 

We will use [`narative-qa dataset`](https://huggingface.co/datasets/narrativeqa_manual) and finetune BigBird for abstractive question answering.


In [77]:
from datasets import load_dataset

In [ ]:
!git clone https://github.com/deepmind/narrativeqa --branch master && !sh narrativeqa/download_stories.sh

In [79]:
dataset = load_dataset("narrativeqa_manual", data_dir="narrativeqa/tmp")
dataset

Using custom data configuration default-data_dir=narrativeqa%2Ftmp
Reusing dataset narrativeqa_manual (/root/.cache/huggingface/datasets/narrativeqa_manual/default-data_dir=narrativeqa%2Ftmp/1.0.0/c57377ffa4fc72b25bf692f6676b140db5a36a7d36a56a891e11274afb40a6ba)


DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [67]:
tr_dataset = dataset["train"].map(lambda x: {"seqlen": (len(x["document"]["summary"]["text"]) + len(x["question"]))//4})
tr_dataset = tr_dataset.map(lambda x: {"q_seqlen": len(x["question"]["text"])//4})
tr_dataset

Dataset({
    features: ['answers', 'document', 'question', 'seqlen'],
    num_rows: 32747
})

In [68]:
# lets decide whether we should use BigBird `block_sparse` attention or `original_full` attention
min(tr_dataset["seqlen"]), sum(tr_dataset["seqlen"])/len(tr_dataset["seqlen"]), max(tr_dataset["seqlen"])

# Since avg seqlen < 1024, we should use `original_full` but for the purpose of demonstarting `block_spare` lets try `block_sparse` attention only

(282, 848.2507710629982, 1511)

In [75]:
# lets see the seqlen of question. This may give some idea about the value of `block_size`
min(tr_dataset["q_seqlen"]), sum(tr_dataset["q_seqlen"])/len(tr_dataset["q_seqlen"]), max(tr_dataset["q_seqlen"])

# lets take `block_size=64`

(2, 11.46517238220295, 43)

In [84]:
from transformers import EncoderDecoderModel

model = EncoderDecoderModel.from_encoder_decoder_pretrained("google/bigbird-roberta-base", "google/bigbird-roberta-base")

When using `BigBirdForCausalLM` as decoder, then `attention_type` must be `original_full`. Setting `attention_type=original_full`
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForCausalLM were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.